# Step 1: Pre-processing - Cell-by-Cell Code

##  Import Required Libraries

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import re
import warnings
warnings.filterwarnings('ignore')

print("Libraries imported successfully!")


Libraries imported successfully!


## Load All CSV Files

In [2]:
# Load all 12 CSV files
property_basic = pd.read_csv('./raw_data/1 - Propery_Basic.csv')
property_address = pd.read_csv('./raw_data/2 - Property_Address.csv')
property_features = pd.read_csv('./raw_data/3 - Property_Features.csv')
picture = pd.read_csv('./raw_data/4 - Picture.csv')
sa1_statistics = pd.read_csv('./raw_data/5 - SA1_Satistics.csv')
mapping_sa1 = pd.read_csv('./raw_data/6 - Mapping_SA1.csv')
school = pd.read_csv('./raw_data/7 - School .csv')
school_ranking = pd.read_csv('./raw_data/8 - School ranking.csv')
mapping_school = pd.read_csv('./raw_data/9 - Mapping_School.csv')
train_station = pd.read_csv('./raw_data/10 - Train_Station.csv')
train_time = pd.read_csv('./raw_data/11 - Train_Time.csv')
mapping_train = pd.read_csv('./raw_data/12 - Mapping_Train_Station.csv')

print("All files loaded successfully!")
print(f"Property Basic shape: {property_basic.shape}")
print(f"Property Address shape: {property_address.shape}")
print(f"Property Features shape: {property_features.shape}")


All files loaded successfully!
Property Basic shape: (53220, 13)
Property Address shape: (52918, 7)
Property Features shape: (53204, 36)


## Explore and Check Data Quality

In [3]:
# Check for missing values in key datasets
print("=== Missing Values in Property Basic ===")
print(property_basic.isnull().sum())
print("\n=== Missing Values in Property Address ===")
print(property_address.isnull().sum())
print("\n=== Data Types ===")
print(property_basic.dtypes)


=== Missing Values in Property Basic ===
ID             0
address        0
price          0
bedroom        0
bathroom       0
parking        0
proType        0
sold_date      0
agency_name    0
agency_addr    0
des_head       0
des_content    0
features       0
dtype: int64

=== Missing Values in Property Address ===
ID                  0
Lat                 0
Lng                 0
Formated_Address    0
Locality            0
State               0
Postal Code         0
dtype: int64

=== Data Types ===
ID              int64
address        object
price           int64
bedroom         int64
bathroom        int64
parking         int64
proType        object
sold_date      object
agency_name    object
agency_addr    object
des_head       object
des_content    object
features       object
dtype: object


## Clean Property Basic Data

In [4]:
# Clean property_basic dataset
property_basic_clean = property_basic.copy()

# Convert sold_date to datetime
property_basic_clean['sold_date'] = pd.to_datetime(property_basic_clean['sold_date'], errors='coerce')

# Extract temporal features
property_basic_clean['sold_year'] = property_basic_clean['sold_date'].dt.year
property_basic_clean['sold_month'] = property_basic_clean['sold_date'].dt.month
property_basic_clean['sold_quarter'] = property_basic_clean['sold_date'].dt.quarter

# Handle missing values in numerical columns
property_basic_clean['bedroom'] = property_basic_clean['bedroom'].fillna(property_basic_clean['bedroom'].median())
property_basic_clean['bathroom'] = property_basic_clean['bathroom'].fillna(property_basic_clean['bathroom'].median())
property_basic_clean['parking'] = property_basic_clean['parking'].fillna(0)

# Handle missing values in categorical columns
property_basic_clean['proType'] = property_basic_clean['proType'].fillna('Unknown')

# Fill missing text fields with empty string
text_columns = ['address', 'des_head', 'des_content', 'features', 'agency_name', 'agency_addr']
for col in text_columns:
    if col in property_basic_clean.columns:
        property_basic_clean[col] = property_basic_clean[col].fillna('')

print("Property Basic data cleaned!")
print(f"Shape: {property_basic_clean.shape}")
print(f"Date range: {property_basic_clean['sold_date'].min()} to {property_basic_clean['sold_date'].max()}")


Property Basic data cleaned!
Shape: (53220, 16)
Date range: 2013-01-02 00:00:00 to 2015-10-30 00:00:00


##  Clean Property Address Data

In [5]:
# Clean property_address dataset
property_address_clean = property_address.copy()

# Handle missing coordinates
property_address_clean['Lat'] = property_address_clean['Lat'].fillna(property_address_clean['Lat'].median())
property_address_clean['Lng'] = property_address_clean['Lng'].fillna(property_address_clean['Lng'].median())

# Fill missing text fields
for col in ['Formated_Address', 'Locality', 'State', 'Postal Code']:
    if col in property_address_clean.columns:
        property_address_clean[col] = property_address_clean[col].fillna('')

print("Property Address data cleaned!")
print(f"Shape: {property_address_clean.shape}")


Property Address data cleaned!
Shape: (52918, 7)


## Clean Property Features Data

In [6]:
# Clean property_features dataset
property_features_clean = property_features.copy()

# Get all feature columns (excluding ID)
feature_columns = [col for col in property_features_clean.columns if col != 'ID']

# Fill missing values with 0 (assuming absence of feature)
for col in feature_columns:
    property_features_clean[col] = property_features_clean[col].fillna(0)
    # Convert to binary (0 or 1)
    property_features_clean[col] = property_features_clean[col].apply(lambda x: 1 if x > 0 else 0)

# Count total amenities per property
property_features_clean['total_amenities'] = property_features_clean[feature_columns].sum(axis=1)

print("Property Features data cleaned!")
print(f"Shape: {property_features_clean.shape}")
print(f"Total feature columns: {len(feature_columns)}")
print(f"Average amenities per property: {property_features_clean['total_amenities'].mean():.2f}")


Property Features data cleaned!
Shape: (53204, 37)
Total feature columns: 35
Average amenities per property: 7.10


## Clean SA1 Statistics Data

In [7]:
# Clean sa1_statistics dataset
sa1_statistics_clean = sa1_statistics.copy()

# Handle missing values in numerical columns with median
numerical_cols = sa1_statistics_clean.select_dtypes(include=[np.number]).columns.tolist()
numerical_cols = [col for col in numerical_cols if col not in ['region_id', 'SA1_ID']]

for col in numerical_cols:
    sa1_statistics_clean[col] = sa1_statistics_clean[col].fillna(sa1_statistics_clean[col].median())

print("SA1 Statistics data cleaned!")
print(f"Shape: {sa1_statistics_clean.shape}")
print(f"Numerical columns cleaned: {len(numerical_cols)}")


SA1 Statistics data cleaned!
Shape: (13339, 14)
Numerical columns cleaned: 12


## Clean School Data

In [8]:
# Clean school dataset
school_clean = school.copy()

# Handle missing values
school_clean['gender'] = school_clean['gender'].fillna('Coed')
school_clean['restrictedZone'] = school_clean['restrictedZone'].fillna(0)
school_clean['type'] = school_clean['type'].fillna('Unknown')
school_clean['name'] = school_clean['name'].fillna('')

# Handle coordinates
school_clean['Lat'] = school_clean['Lat'].fillna(school_clean['Lat'].median())
school_clean['Lng'] = school_clean['Lng'].fillna(school_clean['Lng'].median())

print("School data cleaned!")
print(f"Shape: {school_clean.shape}")


School data cleaned!
Shape: (709, 7)


## Clean School Ranking Data

In [17]:
# Clean school_ranking dataset
school_ranking_clean = school_ranking.copy()

print(school_ranking_clean.columns.tolist())

['school_ID', 'oriName', 'Ranking ', 'Locality', 'IB', 'Students Enrolled in VCE', 'Median VCE score', 'Scores of 40+ (%)']


In [18]:
school_ranking_clean.columns = school_ranking_clean.columns.str.strip()
print(school_ranking_clean.columns.tolist())

['school_ID', 'oriName', 'Ranking', 'Locality', 'IB', 'Students Enrolled in VCE', 'Median VCE score', 'Scores of 40+ (%)']


In [19]:
# Handle missing values in rankings and scores
school_ranking_clean['Ranking'] = school_ranking_clean['Ranking'].fillna(999)  # Low rank for missing
school_ranking_clean['IB'] = school_ranking_clean['IB'].fillna(0)
school_ranking_clean['Students Enrolled in VCE'] = school_ranking_clean['Students Enrolled in VCE'].fillna(0)
school_ranking_clean['Median VCE score'] = school_ranking_clean['Median VCE score'].fillna(school_ranking_clean['Median VCE score'].median())
school_ranking_clean['Scores of 40+ (%)'] = school_ranking_clean['Scores of 40+ (%)'].fillna(0)

print("School Ranking data cleaned!")
print(f"Shape: {school_ranking_clean.shape}")


School Ranking data cleaned!
Shape: (191, 8)


## Clean Train Station Data

In [20]:
# Clean train_station dataset
train_station_clean = train_station.copy()

# Handle missing values
train_station_clean['stop_name'] = train_station_clean['stop_name'].fillna('')
train_station_clean['stop_short_name'] = train_station_clean['stop_short_name'].fillna('')
train_station_clean['Lat'] = train_station_clean['Lat'].fillna(train_station_clean['Lat'].median())
train_station_clean['Lng'] = train_station_clean['Lng'].fillna(train_station_clean['Lng'].median())

print("Train Station data cleaned!")
print(f"Shape: {train_station_clean.shape}")


Train Station data cleaned!
Shape: (218, 6)


## Clean Mapping Tables

In [21]:
# Clean mapping tables - these should generally have no missing values
# but we'll check and handle them

# Mapping SA1
mapping_sa1_clean = mapping_sa1.dropna()

# Mapping School
mapping_school_clean = mapping_school.dropna()

# Mapping Train Station
mapping_train_clean = mapping_train.copy()
mapping_train_clean['distance_value1'] = mapping_train_clean['distance_value1'].fillna(mapping_train_clean['distance_value1'].median())
mapping_train_clean['duration_value1'] = mapping_train_clean['duration_value1'].fillna(mapping_train_clean['duration_value1'].median())

print("Mapping tables cleaned!")
print(f"Mapping SA1 shape: {mapping_sa1_clean.shape}")
print(f"Mapping School shape: {mapping_school_clean.shape}")
print(f"Mapping Train shape: {mapping_train_clean.shape}")


Mapping tables cleaned!
Mapping SA1 shape: (52914, 2)
Mapping School shape: (52913, 2)
Mapping Train shape: (52913, 7)


## Prepare Data for TEXT EMBEDDINGS

In [31]:
# ===== TEXT EMBEDDINGS DATA =====
# Combine all textual information for each property

text_embedding_data = property_basic_clean[['ID', 'address', 'des_head', 'des_content', 'features']].copy()

# Merge with address information
text_embedding_data = text_embedding_data.merge(
    property_address_clean[['ID', 'Formated_Address', 'Locality', 'State', 'Postal Code']], 
    on='ID', 
    how='left'
)

# Add agency information
text_embedding_data['agency_name'] = property_basic_clean['agency_name']

# Create combined text field for embedding
text_embedding_data['combined_text'] = (
    text_embedding_data['des_head'].astype(str) + ' ' +
    text_embedding_data['des_content'].astype(str) + ' ' +
    text_embedding_data['features'].astype(str) + ' ' +
    text_embedding_data['Locality'].astype(str) + ' ' +
    text_embedding_data['State'].astype(str)
)

# Clean combined text
text_embedding_data['combined_text'] = text_embedding_data['combined_text'].str.strip()

# Save for text embeddings
text_embedding_data.to_csv('./Data/Text/preprocessed_text_embeddings.csv', index=False)

print("✓ Text Embedding Data Prepared!")
print(f"Shape: {text_embedding_data.shape}")
print(f"Columns: {list(text_embedding_data.columns)}")
print(f"\nSample combined text:")
print(text_embedding_data['combined_text'].iloc[0][:200], "...")


✓ Text Embedding Data Prepared!
Shape: (53220, 11)
Columns: ['ID', 'address', 'des_head', 'des_content', 'features', 'Formated_Address', 'Locality', 'State', 'Postal Code', 'agency_name', 'combined_text']

Sample combined text:
Farmlet Makeover Established farmlet that has had a major makeover with new kitchen, bathroom, en-suite and family room.  Located only 16Kms from Sale a peaceful rural location is this established pro ...


## Prepare Data for IMAGE EMBEDDINGS

In [32]:
# ===== IMAGE EMBEDDINGS DATA =====
# Prepare image metadata with property information

image_embedding_data = picture.copy()

# Sort by ID and picNo to maintain order
image_embedding_data = image_embedding_data.sort_values(['ID', 'picNo'])

# Add property price for reference (target variable)
image_embedding_data = image_embedding_data.merge(
    property_basic_clean[['ID', 'price', 'proType']], 
    on='ID', 
    how='left'
)

# Add location information
image_embedding_data = image_embedding_data.merge(
    property_address_clean[['ID', 'Locality']], 
    on='ID', 
    how='left'
)

# Save for image embeddings
image_embedding_data.to_csv('./Data/Image/preprocessed_image_embeddings.csv', index=False)

print("✓ Image Embedding Data Prepared!")
print(f"Shape: {image_embedding_data.shape}")
print(f"Columns: {list(image_embedding_data.columns)}")
print(f"Total properties with images: {image_embedding_data['ID'].nunique()}")
print(f"Average images per property: {image_embedding_data.groupby('ID').size().mean():.2f}")


✓ Image Embedding Data Prepared!
Shape: (262179, 6)
Columns: ['ID', 'picNo', 'picAddr', 'price', 'proType', 'Locality']
Total properties with images: 53220
Average images per property: 4.93


## Prepare Data for GRAPH EMBEDDINGS - Part 1 (Node Features)

In [24]:
# ===== GRAPH EMBEDDINGS DATA =====
# Prepare comprehensive graph structure with nodes and edges

# Start with core property features
graph_nodes_properties = property_basic_clean[[
    'ID', 'bedroom', 'bathroom', 'parking', 'proType', 
    'sold_year', 'sold_month', 'sold_quarter', 'price'
]].copy()

# Add location features
graph_nodes_properties = graph_nodes_properties.merge(
    property_address_clean[['ID', 'Lat', 'Lng', 'Locality', 'State', 'Postal Code']], 
    on='ID', 
    how='left'
)

# Add all amenity features
graph_nodes_properties = graph_nodes_properties.merge(
    property_features_clean, 
    on='ID', 
    how='left'
)

# Add SA1 statistics
graph_nodes_properties = graph_nodes_properties.merge(
    mapping_sa1_clean, 
    on='ID', 
    how='left'
)

graph_nodes_properties = graph_nodes_properties.merge(
    sa1_statistics_clean, 
    on='SA1_ID', 
    how='left'
)

print("✓ Graph Node Features (Properties) Prepared!")
print(f"Shape: {graph_nodes_properties.shape}")
print(f"Total features per property: {graph_nodes_properties.shape[1]}")


✓ Graph Node Features (Properties) Prepared!
Shape: (53220, 64)
Total features per property: 64


## Prepare Data for GRAPH EMBEDDINGS - Part 2 (School Nodes)

In [25]:
# Prepare school nodes with rankings
graph_nodes_schools = school_clean.merge(
    school_ranking_clean, 
    on='school_ID', 
    how='left'
)

# Fill missing rankings
graph_nodes_schools['Ranking'] = graph_nodes_schools['Ranking'].fillna(999)
graph_nodes_schools['Median VCE score'] = graph_nodes_schools['Median VCE score'].fillna(
    graph_nodes_schools['Median VCE score'].median()
)

print("✓ Graph Node Features (Schools) Prepared!")
print(f"Shape: {graph_nodes_schools.shape}")
print(f"Columns: {list(graph_nodes_schools.columns)}")


✓ Graph Node Features (Schools) Prepared!
Shape: (709, 14)
Columns: ['school_ID', 'name', 'gender', 'restrictedZone', 'type', 'Lng', 'Lat', 'oriName', 'Ranking', 'Locality', 'IB', 'Students Enrolled in VCE', 'Median VCE score', 'Scores of 40+ (%)']


## Prepare Data for GRAPH EMBEDDINGS - Part 3 (Train Station Nodes)

In [26]:
# Prepare train station nodes
graph_nodes_stations = train_station_clean.copy()

print("✓ Graph Node Features (Train Stations) Prepared!")
print(f"Shape: {graph_nodes_stations.shape}")
print(f"Columns: {list(graph_nodes_stations.columns)}")


✓ Graph Node Features (Train Stations) Prepared!
Shape: (218, 6)
Columns: ['stop_id', 'stop_no', 'stop_short_name', 'stop_name', 'Lat', 'Lng']


## Prepare Data for GRAPH EMBEDDINGS - Part 4 (Property-School Edges)

In [27]:
# Prepare edges: Property to School relationships
graph_edges_property_school = mapping_school_clean.copy()
graph_edges_property_school['edge_type'] = 'property_to_school'

# Add school rankings to edges
graph_edges_property_school = graph_edges_property_school.merge(
    school_ranking_clean[['school_ID', 'Ranking', 'Median VCE score']], 
    on='school_ID', 
    how='left'
)

print("✓ Graph Edges (Property-School) Prepared!")
print(f"Shape: {graph_edges_property_school.shape}")
print(f"Total property-school connections: {len(graph_edges_property_school)}")


✓ Graph Edges (Property-School) Prepared!
Shape: (52913, 5)
Total property-school connections: 52913


## Prepare Data for GRAPH EMBEDDINGS - Part 5 (Property-Station Edges)

In [28]:
# Prepare edges: Property to Train Station relationships
graph_edges_property_station = mapping_train_clean[['ID', 'stop_id', 'distance_value1', 'duration_value1']].copy()
graph_edges_property_station['edge_type'] = 'property_to_station'

# Rename columns for consistency
graph_edges_property_station.rename(columns={
    'distance_value1': 'distance_meters',
    'duration_value1': 'duration_seconds'
}, inplace=True)

print("✓ Graph Edges (Property-Station) Prepared!")
print(f"Shape: {graph_edges_property_station.shape}")
print(f"Total property-station connections: {len(graph_edges_property_station)}")


✓ Graph Edges (Property-Station) Prepared!
Shape: (52913, 5)
Total property-station connections: 52913


## Prepare Data for GRAPH EMBEDDINGS - Part 6 (Station-Station Edges)

In [29]:
# Prepare edges: Train Station to Train Station relationships (connectivity)
graph_edges_station_station = train_time.copy()
graph_edges_station_station['edge_type'] = 'station_to_station'

# Rename columns for consistency
graph_edges_station_station.rename(columns={
    'stop_ori': 'source_station',
    'stop_des': 'target_station',
    'avg_time': 'travel_time_minutes',
    'trans_flag': 'requires_transfer'
}, inplace=True)

print("✓ Graph Edges (Station-Station) Prepared!")
print(f"Shape: {graph_edges_station_station.shape}")
print(f"Total station connections: {len(graph_edges_station_station)}")


✓ Graph Edges (Station-Station) Prepared!
Shape: (47524, 6)
Total station connections: 47524


## Save All Graph Embedding Files

In [30]:
# Save all graph-related files
graph_nodes_properties.to_csv('./Data/Graphs/graph_nodes_properties.csv', index=False)
graph_nodes_schools.to_csv('./Data/Graphs/graph_nodes_schools.csv', index=False)
graph_nodes_stations.to_csv('./Data/Graphs/graph_nodes_stations.csv', index=False)
graph_edges_property_school.to_csv('./Data/Graphs/graph_edges_property_school.csv', index=False)
graph_edges_property_station.to_csv('./Data/Graphs/graph_edges_property_station.csv', index=False)
graph_edges_station_station.to_csv('./Data/Graphs/graph_edges_station_station.csv', index=False)

print("✓ All Graph Embedding Files Saved!")
print("\nGraph Files Created:")
print("  1. graph_nodes_properties.csv")
print("  2. graph_nodes_schools.csv")
print("  3. graph_nodes_stations.csv")
print("  4. graph_edges_property_school.csv")
print("  5. graph_edges_property_station.csv")
print("  6. graph_edges_station_station.csv")


✓ All Graph Embedding Files Saved!

Graph Files Created:
  1. graph_nodes_properties.csv
  2. graph_nodes_schools.csv
  3. graph_nodes_stations.csv
  4. graph_edges_property_school.csv
  5. graph_edges_property_station.csv
  6. graph_edges_station_station.csv


## Create Summary Report

In [34]:
# Create comprehensive preprocessing summary
print("=" * 80)
print("PREPROCESSING SUMMARY - PROPERTY VALUATION PREDICTION SYSTEM")
print("=" * 80)

print("\nDATASET OVERVIEW")
print("-" * 80)
print(f"Total Properties: {property_basic_clean['ID'].nunique()}")
print(f"Price Range: ${property_basic_clean['price'].min():,.0f} - ${property_basic_clean['price'].max():,.0f}")
print(f"Average Price: ${property_basic_clean['price'].mean():,.0f}")
print(f"Median Price: ${property_basic_clean['price'].median():,.0f}")

print("\nTEXT EMBEDDINGS")
print("-" * 80)
print(f"File: preprocessed_text_embeddings.csv")
print(f"Records: {len(text_embedding_data)}")
print(f"Columns: {text_embedding_data.shape[1]}")
print(f"Key Fields: address, des_head, des_content, features, combined_text")

print("\nIMAGE EMBEDDINGS")
print("-" * 80)
print(f"File: preprocessed_image_embeddings.csv")
print(f"Total Images: {len(image_embedding_data)}")
print(f"Properties with Images: {image_embedding_data['ID'].nunique()}")
print(f"Avg Images per Property: {image_embedding_data.groupby('ID').size().mean():.2f}")

print("\nGRAPH EMBEDDINGS")
print("-" * 80)
print(f"Property Nodes: {len(graph_nodes_properties)}")
print(f"School Nodes: {len(graph_nodes_schools)}")
print(f"Station Nodes: {len(graph_nodes_stations)}")
print(f"Property-School Edges: {len(graph_edges_property_school)}")
print(f"Property-Station Edges: {len(graph_edges_property_station)}")
print(f"Station-Station Edges: {len(graph_edges_station_station)}")
print(f"Total Graph Features per Property: {graph_nodes_properties.shape[1]}")

print("\nPREPROCESSING COMPLETE!")
print("=" * 80)
print("\nNext Steps:")
print("  → Step 2: Create Graph Embeddings using graph_*.csv files")
print("  → Step 3: Create Text Embeddings using preprocessed_text_embeddings.csv")
print("  → Step 4: Create Image Embeddings using preprocessed_image_embeddings.csv")
print("  → Step 5: Combine all embeddings and train prediction model")


PREPROCESSING SUMMARY - PROPERTY VALUATION PREDICTION SYSTEM

DATASET OVERVIEW
--------------------------------------------------------------------------------
Total Properties: 53220
Price Range: $11,562 - $123,456,792
Average Price: $707,975
Median Price: $610,000

TEXT EMBEDDINGS
--------------------------------------------------------------------------------
File: preprocessed_text_embeddings.csv
Records: 53220
Columns: 11
Key Fields: address, des_head, des_content, features, combined_text

IMAGE EMBEDDINGS
--------------------------------------------------------------------------------
File: preprocessed_image_embeddings.csv
Total Images: 262179
Properties with Images: 53220
Avg Images per Property: 4.93

GRAPH EMBEDDINGS
--------------------------------------------------------------------------------
Property Nodes: 53220
School Nodes: 709
Station Nodes: 218
Property-School Edges: 52913
Property-Station Edges: 52913
Station-Station Edges: 47524
Total Graph Features per Property: 